In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/churn-modelling/Churn_Modelling.csv


In [2]:
df = pd.read_csv("../input/churn-modelling/Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
missing_cols = [col for col in df.columns if df[col].isnull().any()]
missing_cols

[]

In [4]:
# Categorical columns
df_categorical = df.select_dtypes('object')

In [5]:
from sklearn.preprocessing import LabelEncoder
df_c = df.copy()
label_encoder = LabelEncoder()
for col in df_categorical.columns:
    df_c[col] = label_encoder.fit_transform(df[col])

In [6]:
y = df_c[['Exited']]
df_c.drop('Exited', axis=1, inplace=True)
X = df_c

In [7]:
from sklearn.model_selection import train_test_split
X_train_c, X_test_c, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
from sklearn.preprocessing import MinMaxScaler
# Standardization
standard_scaler = MinMaxScaler(feature_range=(0, 1))
X_train = pd.DataFrame(standard_scaler.fit_transform(X_train_c))
X_test = pd.DataFrame(standard_scaler.fit_transform(X_test_c))
print(X_train.head())

          0         1         2      3    4    5         6    7         8  \
0  0.258526  0.185330  0.911975  0.748  0.0  1.0  0.135135  0.9  0.000000   
1  0.858186  0.058039  0.597407  0.470  0.0  0.0  0.337838  0.2  0.000000   
2  0.334533  0.330517  0.393381  0.616  0.0  0.0  0.175676  0.7  0.494125   
3  0.565457  0.727142  0.310133  0.654  1.0  0.0  0.337838  0.3  0.530952   
4  0.519952  0.642534  0.628796  0.864  1.0  0.0  0.121622  0.3  0.000000   

          9   10   11        12  
0  0.333333  1.0  1.0  0.501191  
1  0.333333  1.0  0.0  0.446997  
2  0.000000  1.0  0.0  0.510760  
3  0.333333  1.0  1.0  0.479672  
4  0.333333  1.0  0.0  0.718081  


In [9]:
X_train.shape[1]

13

In [10]:
import keras as keras
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[X_train.shape[1]]))
model.add(keras.layers.Dense(100, kernel_initializer="normal", activation="relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(50, kernel_initializer="normal", activation="relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(2, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [11]:
history=model.fit(X_train, y_train, epochs=100)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5056 - accuracy: 0.7644
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4009 - accuracy: 0.8314
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3854 - accuracy: 0.8361
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3791 - accuracy: 0.8424
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3730 - accuracy: 0.8431
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3692 - accuracy: 0.8445
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3607 - accuracy: 0.8524
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3602 - accuracy: 0.8501
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3599 - accuracy: 0.8501
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3538 - accura

In [12]:
model.evaluate(X_test, y_test)

63/63 [==============================] - 0s 1ms/step - loss: 0.4300 - accuracy: 0.8270


[0.4299543797969818, 0.8270000219345093]